In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input

In [2]:
names = ["Anudari", "Cebastian", "Chris", "Michael"]
mask_label = ['with mask', 'without mask']

In [3]:
####################
### MASK/NO MASK ###
####################
mask_no_mask_model = tf.keras.models.load_model('../converging_models/MODELS/mask_nomask_model.h5')
#mask_no_mask_model.summary()

#####################
### UNMASKED FACE ###
#####################
unmasked_model = tf.keras.models.load_model('../converging_models/MODELS/new_nomask_model.h5')
#unmasked_model.summary()

###################
### MASKED FACE ###
###################
masked_model = tf.keras.models.load_model('../converging_models/MODELS/new_mask_model.h5')
#masked_model.summary()

######################
### LOAD DNN MODEL ###
######################
network = cv2.dnn.readNetFromCaffe("../converging_models/MODELS/deploy.prototxt", "../converging_models/MODELS/res10_300x300_ssd_iter_140000_fp16.caffemodel")

In [38]:
##################
### LOAD IMAGE ###
##################

#image_path = ('../../../../ElitePictures/Home/received_2000238823583079.jpeg')
image_path = ('../NEW_DATASET/test/IMG_0488.JPG')

############################
### LABEL MULTIPLE FACES ###
############################
dnn_label = ''
scale_factor = 0.3
confidence_threshold = 0.85
dnn_boxColor = (0,0,0)

dnn_image = cv2.imread(image_path)
#dnn_image = cv2.resize(dnn_image, (582,448))
dnn_image = cv2.resize(dnn_image, (0,0), fx=scale_factor, fy=scale_factor)


(height, width) = dnn_image.shape[:2]
blob = cv2.dnn.blobFromImage(dnn_image, scalefactor=1.0, size=(300, 300), mean=(104.0, 117.0, 123.0))
network.setInput(blob)
detections = network.forward()

for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    if confidence > 0.5:
        box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
        left = int(box[0])
        top = int(box[1])
        right = int(box[2])
        bottom = int(box[3])

        cropped_image = dnn_image[top:bottom, left:right]

        if cropped_image.size < 1:
            continue

        ###
        new_frame = cv2.resize(cropped_image, (224,224))
        x = image.img_to_array(new_frame)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        dnn_prediction = ''

        mask_prediction = mask_no_mask_model.predict(x)

        if mask_label[np.argmax(mask_prediction, axis=1)[0]] == 'with mask':
            dnn_label = 'Masked - '
            dnn_prediction = masked_model.predict(x)
            dnn_boxColor = (255,0,0)
        else:
            dnn_label = 'Unmasked - '
            dnn_prediction = unmasked_model.predict(x)
            dnn_boxColor = (0,0,255)

        prediction_prob = dnn_prediction[0][np.argmax(dnn_prediction, axis=1)[0]]

        if prediction_prob < confidence_threshold:
            dnn_label = dnn_label + "Unknown"
        else :
            dnn_label = dnn_label + names[np.argmax(dnn_prediction, axis=1)[0]]
        ###

        cv2.rectangle(dnn_image, (left - 10, top - 50), (right + 10, bottom + 50), dnn_boxColor, 2)
        cv2.rectangle(dnn_image, (left - 11, top - 49), (right + 11, top -24), dnn_boxColor, cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(dnn_image, dnn_label, (left - 6, top - 36), font, 0.5, (255, 255, 255), 1)

''' Uncomment next line to save image '''
cv2.imwrite('../NEW_DATASET/test/masks_off_threshold.jpg', dnn_image)

cv2.imshow("DNN", dnn_image)
cv2.waitKey(1)

1/1 [==============================] - 0s 22ms/step


-1

In [68]:
######################
### VIDEO WITH DNN ###
######################
predictionLabel = ''
confidence_threshold = 0.85
scale_factor = 0.5
boxColor = (0,0,0)

#video = cv2.VideoCapture(0)
video = cv2.VideoCapture("../NEW_DATASET/test/IMG_0485.MOV")
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width // 2, frame_height // 2)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
new_video = cv2.VideoWriter("../NEW_DATASET/test/group_with_boxes.avi", fourcc, 15.0, size)



if (video.isOpened() == False):
    print("Web Camera not detected")
while (True):
    ret, frame = video.read()
    
    if ret == True:
        frame = cv2.flip(frame, 0)

        ### DETECT FACE LOCATION USING DNN
        dnn_label = ''
        dnn_boxColor = (0,0,0)

        #dnn_image = cv2.resize(frame, (582,448))
        dnn_image = cv2.resize(frame, (0,0), fx=scale_factor, fy=scale_factor)

        (height, width) = dnn_image.shape[:2]
        blob = cv2.dnn.blobFromImage(dnn_image, scalefactor=1.0, size=(300, 300), mean=(104.0, 117.0, 123.0))
        network.setInput(blob)
        detections = network.forward()
        

        for i in range(0, detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([width, height, width, height])
                left = int(box[0])
                top = int(box[1])
                right = int(box[2])
                bottom = int(box[3])

                cropped_image = dnn_image[top:bottom, left:right]

                if cropped_image.size < 1:
                    continue

                ###
                new_frame = cv2.resize(cropped_image, (224,224))
                x = image.img_to_array(new_frame)
                x = np.expand_dims(x, axis=0)
                x = preprocess_input(x)

                dnn_prediction = ''

                mask_prediction = mask_no_mask_model.predict(x)

                if mask_label[np.argmax(mask_prediction, axis=1)[0]] == 'with mask':
                    dnn_label = 'Masked - '
                    dnn_prediction = masked_model.predict(x)
                    dnn_boxColor = (255,0,0)
                else:
                    dnn_label = 'Unmasked - '
                    dnn_prediction = unmasked_model.predict(x)
                    dnn_boxColor = (0,0,255)

                prediction_prob = dnn_prediction[0][np.argmax(dnn_prediction, axis=1)[0]]

                if prediction_prob < confidence_threshold:
                    dnn_label = dnn_label + "Unknown"
                else :
                    dnn_label = dnn_label + names[np.argmax(dnn_prediction, axis=1)[0]]
                ###

                cv2.rectangle(dnn_image, (left - 10, top - 50), (right + 10, bottom + 50), dnn_boxColor, 2)
                cv2.rectangle(dnn_image, (left - 11, top - 49), (right + 11, top -24), dnn_boxColor, cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(dnn_image, dnn_label, (left - 6, top - 36), font, 0.5, (255, 255, 255), 1)

        if dnn_image.size < 1:
            continue
        new_video.write(dnn_image)
        cv2.imshow("DNN", dnn_image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
new_video.release()
video.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 44ms/step
